In [1]:
import pandas as pd

In [2]:
ProbNonDefaultGivenScoreGreaterXAndRace = pd.read_csv("ProbNonDefaultGivenScoreGreaterXAndRace.csv")
ProbLoanReceiverIsGood = pd.read_csv("ProbLoanReceiverIsGood.csv")

# ProbOfBeingScore = ProbScoreEqualsXGivenRace

In [5]:
ProbOfBeingScore = (
    pd.read_csv("figure3A.csv")
    .set_index(["Score"]).diff().fillna(0) / 100
)
# ProbOfBeingScore.columns.name = "Demographic"
ProbOfBeingScore.columns = ProbOfBeingScore.columns.str.lower()
ProbOfBeingScore.index.name = "TransRisk Score"

In [10]:
ProbScoreEqualsXGivenRace = pd.read_csv("ProbScoreEqualsXGivenRace.csv").set_index("Score")

In [15]:
for race in ["White", "Black", "Hispanic", "Asian"]:
    print(race, (ProbOfBeingScore[race.lower()] - ProbScoreEqualsXGivenRace[race]).abs().sum())

White 5.692061405548898e-17
Black 4.7488055154865094e-17
Hispanic 5.800481622797449e-17
Asian 5.117434254131581e-17


# ProbOfBeingGoodAtScore = ProbNonDefaultAndScoreEqualsXGivenRace

In [21]:
ProbOfBeingGoodAtScore = (
    pd.read_csv("PercentageOfGoodsAndBads.csv")
    .set_index(["TransRisk Score", "Demographic"])
    .unstack()["Good"]
)

In [20]:
pis = [0.759185,0.315164,0.550595,0.80066]
CumulativePercentageByDefaulters = pd.read_csv("Figure7A-fixed.csv")

sizes = [133165, 18274, 14702, 7906]
total = sum(sizes)
ProbOfBeingRace = pd.DataFrame(
    { 
    'Demographic' : ['white', 'black', 'hispanic', 'asian'],
    'P(race)' : [sizes[0]/total, sizes[1]/total, sizes[2]/total, sizes[3]/total]
    },
    columns=["Demographic", "P(race)"]
)
ProbOfBeingRace.set_index('Demographic', inplace=True)
ProbRaceAndScoreEqualsX = pd.DataFrame({
    'white': ProbOfBeingRace.loc['white']['P(race)'] * ProbScoreEqualsXGivenRace['White'],
    'asian': ProbOfBeingRace.loc['asian']['P(race)'] * ProbScoreEqualsXGivenRace['Asian'],
    'black': ProbOfBeingRace.loc['black']['P(race)'] * ProbScoreEqualsXGivenRace['Black'],
    'hispanic': ProbOfBeingRace.loc['hispanic']['P(race)'] * ProbScoreEqualsXGivenRace['Hispanic'],
})

def getProbDefOrNotGivenScoreEqualsX(dataset, names, pis):
    probabilities = pd.DataFrame(index=dataset.index)
    for i in range(len(names)):
        nonDefault = dataset[names[i] + ' (NonDefault)'].diff().fillna(value=0)
        default = dataset[names[i] + ' (Default)'].diff().fillna(value=0)
        finalProbability = (
            (pis[i] * nonDefault) / ((pis[i] * nonDefault) + ((1 - pis[i])*(default))))
        probabilities['P(NonDefault|Score=x, ' + names[i] + ')'] = finalProbability.fillna(value=0)
        probabilities['P(Default|Score=x, ' + names[i] + ')'] = (1 - finalProbability.fillna(value=0))
    probabilities['Score'] = dataset.index
    return probabilities.set_index('Score')

ProbDefOrNotGivenScoreEqualsXAndRace = getProbDefOrNotGivenScoreEqualsX(
    CumulativePercentageByDefaulters.set_index('Score'),
    ["White", "Black", "Hispanic", "Asian"],
    pis)

ProbNonDefaultGivenScoreEqualsXAndRace = (
    ProbDefOrNotGivenScoreEqualsXAndRace[['P(NonDefault|Score=x, White)', 
                                 'P(NonDefault|Score=x, Black)',
                                'P(NonDefault|Score=x, Hispanic)',
                                'P(NonDefault|Score=x, Asian)']])

ProbScoreEqualsXAndGoodAndRace = pd.DataFrame(index=ProbRaceAndScoreEqualsX.index)
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and White)'] = (
    ProbRaceAndScoreEqualsX['white'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, White)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Black)'] = (
    ProbRaceAndScoreEqualsX['black'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Black)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Hispanic)'] = (
    ProbRaceAndScoreEqualsX['hispanic'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Hispanic)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Asian)'] = (
    ProbRaceAndScoreEqualsX['asian'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Asian)'])

ProbNonDefaultGivenScoreEqualsXAndRace.columns= ['Asian', 'Black', 'Hispanic', 'White']

In [26]:
for race in ["White", "Black", "Hispanic", "Asian"]:
    print(race, (ProbOfBeingGoodAtScore[race.lower()] - ProbNonDefaultGivenScoreEqualsXAndRace[race]).abs().sum())

White 8.502342054140064
Black 1.6674386454431702
Hispanic 0.4350214440731871
Asian 8.92581709234939
